Import necessary libraries

In [1]:
import os
import json
import pandas as pd

Code to basic functionalities used later on in the code

In [2]:
def load_json(file_name):
    with open(file_name, encoding='utf-8') as f:
        iter_f = iter(f)
        line = f.readline()
        list_name = []
        for line in iter_f:
            d = json.loads(line)
            list_name.append(d)
        f.close()
    return pd.DataFrame(list_name)

Change directory to folder with the JSON files from "Yelp Data"

In [3]:
os.chdir("C:/Users/lokes/Documents/Files/Stat628")

Load JSON files as dataframes

In [8]:
business = load_json('business.json')
review = load_json('review.json')
#tip = load_json('tip.json')
#user = load_json('user.json')

In [7]:
business.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')

In [79]:
business = business[business.is_open == 1]
state_counts = business.groupby('state').count()
state_counts['business_id'].sort_values(ascending = False).head(10)

state
AZ    46910
NV    29562
ON    26525
OH    12546
NC    12419
PA     9430
QC     7623
AB     6694
WI     4210
IL     1545
Name: business_id, dtype: int64

In [ ]:
As we can see based on geography, 3 of top 10 states come from Canada - ON,QC,AB.
We would like to focus our analysis based on only states from Canada.

In [80]:
state_list = ['ON','QC','AB']
business1 = business[business.state.isin(state_list)]
business1.shape

(40842, 14)

In [117]:
category_count = business1.groupby('categories').count()
category_count['state'].sort_values(ascending = False).head(20)

categories
Food, Coffee & Tea            325
Pizza, Restaurants            312
Restaurants, Pizza            309
Coffee & Tea, Food            306
Beauty & Spas, Hair Salons    268
Hair Salons, Beauty & Spas    266
Chinese, Restaurants          262
Restaurants, Chinese          238
Nail Salons, Beauty & Spas    212
Food, Grocery                 195
Grocery, Food                 187
Beauty & Spas, Nail Salons    185
Restaurants, Italian          154
Bakeries, Food                152
Restaurants, Vietnamese       151
Vietnamese, Restaurants       150
Italian, Restaurants          149
Food, Bakeries                144
Indian, Restaurants           124
Automotive, Auto Repair       118
Name: state, dtype: int64

From above table, it so appears that Chinese cuisine is the most popular category.
We therefore limit our analysis to include only restaurants which serve chinese cuisine.

Lets check if the number of data points as limited to chinese cuisine in ON,QC,AB are good enough for us?

In [124]:
business2 = business1[business1['categories'].str.contains('chinese',case = False).fillna(False)]
business2.shape

(1593, 14)

In [125]:
business.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')

In [126]:
review.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')

In [140]:
canada = pd.merge(business2[business2.review_count >= 10] , review , on = 'business_id')
canada.shape

(62233, 22)

In [145]:
canada.groupby('state').business_id.nunique().sum()

1066

    State: ON,QC,AB
    Category: contains 'chinese' 

As we can see, in our chosen category, there are 1066 restaurants(atleast 10 reviews each) with a review count of 62233.